In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hg14-handgesture14-dataset/HG14/Read.docx
/kaggle/input/hg14-handgesture14-dataset/HG14/HG14-Hand Gesture/Gesture_11/11925.jpg
/kaggle/input/hg14-handgesture14-dataset/HG14/HG14-Hand Gesture/Gesture_11/11378.jpg
/kaggle/input/hg14-handgesture14-dataset/HG14/HG14-Hand Gesture/Gesture_11/11138.jpg
/kaggle/input/hg14-handgesture14-dataset/HG14/HG14-Hand Gesture/Gesture_11/11868.jpg
/kaggle/input/hg14-handgesture14-dataset/HG14/HG14-Hand Gesture/Gesture_11/11034.jpg
/kaggle/input/hg14-handgesture14-dataset/HG14/HG14-Hand Gesture/Gesture_11/11251.jpg
/kaggle/input/hg14-handgesture14-dataset/HG14/HG14-Hand Gesture/Gesture_11/11617.jpg
/kaggle/input/hg14-handgesture14-dataset/HG14/HG14-Hand Gesture/Gesture_11/11921.jpg
/kaggle/input/hg14-handgesture14-dataset/HG14/HG14-Hand Gesture/Gesture_11/11236.jpg
/kaggle/input/hg14-handgesture14-dataset/HG14/HG14-Hand Gesture/Gesture_11/11522.jpg
/kaggle/input/hg14-handgesture14-dataset/HG14/HG14-Hand Gesture/Gesture_11/11529.jpg
/kaggle/i

# 1. ResNet-50 Model 

In [38]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [39]:
def load_images_kaggle():
    images = []
    labels = []

    root_path = '/kaggle/input/hg14-handgesture14-dataset/HG14/'

    for label in range(14):  
        gesture_folder = f'Gesture_{label}'
        gesture_path = os.path.join(root_path, 'HG14-Hand Gesture', gesture_folder)
        
        for dirname, _, filenames in os.walk(gesture_path):
            for filename in filenames:
                if filename.endswith('.jpg'):  
                    filepath = os.path.join(dirname, filename)
                    images.append(filepath)
                    labels.append(label)

    data = pd.DataFrame({'Image': images, 'Label': labels})
    return data

In [40]:
data = load_images_kaggle()

In [41]:
train_df, val_df = train_test_split(data, test_size=0.2, random_state=2024)

In [43]:
def preprocess_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (224, 224))  
    img = tf.keras.applications.resnet50.preprocess_input(img)
    return img

In [44]:
def create_dataset(df, batch_size=32):
    image_paths = df['Image'].values
    labels = df['Label'].values

    image_dataset = tf.data.Dataset.from_tensor_slices(image_paths)
    image_dataset = image_dataset.map(preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)

    label_dataset = tf.data.Dataset.from_tensor_slices(labels)

    dataset = tf.data.Dataset.zip((image_dataset, label_dataset))
    dataset = dataset.shuffle(buffer_size=len(df))
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    return dataset

In [45]:
batch_size = 32
train_dataset = create_dataset(train_df, batch_size=batch_size)
val_dataset = create_dataset(val_df, batch_size=batch_size)

In [46]:
resnet50 = tf.keras.applications.ResNet50(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
for layer in resnet50.layers:
    layer.trainable = False

model = models.Sequential([
    resnet50,
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation='relu'),
    layers.Dense(14, activation='softmax')
])

In [47]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [48]:
epochs = 10
history = model.fit(train_dataset,
                    epochs=epochs,
                    validation_data=val_dataset)

Epoch 1/10
  2/350 ━━━━━━━━━━━━━━━━━━━━ 35s 101ms/step - accuracy: 0.1328 - loss: 3.0080  

W0000 00:00:1710691019.817902     160 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


350/350 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - accuracy: 0.5758 - loss: 1.3231

W0000 00:00:1710691057.530276     160 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


350/350 ━━━━━━━━━━━━━━━━━━━━ 79s 134ms/step - accuracy: 0.5762 - loss: 1.3217 - val_accuracy: 0.8775 - val_loss: 0.3438
Epoch 2/10


W0000 00:00:1710691066.551900     159 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


350/350 ━━━━━━━━━━━━━━━━━━━━ 57s 115ms/step - accuracy: 0.9086 - loss: 0.2744 - val_accuracy: 0.9057 - val_loss: 0.2628
Epoch 3/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 57s 115ms/step - accuracy: 0.9463 - loss: 0.1686 - val_accuracy: 0.9150 - val_loss: 0.2216
Epoch 4/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 57s 117ms/step - accuracy: 0.9607 - loss: 0.1162 - val_accuracy: 0.9432 - val_loss: 0.1679
Epoch 5/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 57s 116ms/step - accuracy: 0.9767 - loss: 0.0787 - val_accuracy: 0.9454 - val_loss: 0.1537
Epoch 6/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 57s 115ms/step - accuracy: 0.9771 - loss: 0.0721 - val_accuracy: 0.9536 - val_loss: 0.1420
Epoch 7/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 56s 115ms/step - accuracy: 0.9781 - loss: 0.0676 - val_accuracy: 0.9296 - val_loss: 0.2019
Epoch 8/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 56s 115ms/step - accuracy: 0.9747 - loss: 0.0704 - val_accuracy: 0.9589 - val_loss: 0.1195
Epoch 9/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 56s 116ms/step - accuracy: 0.9831 - loss: 0.0532 - val

In [49]:
val_loss, val_accuracy = model.evaluate(val_dataset)
formatted_accuracy = "{:.2f}%".format(val_accuracy * 100)
print("Accuracy on validation set:", formatted_accuracy)

88/88 ━━━━━━━━━━━━━━━━━━━━ 11s 81ms/step - accuracy: 0.9503 - loss: 0.1560
Accuracy on validation set: 94.75%


# 2. Improving Model Performance 

## 2.1 Data Augmentation 

In [14]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split

In [15]:
def load_images_kaggle():
    images = []
    labels = []

    root_path = '/kaggle/input/hg14-handgesture14-dataset/HG14/'

    for label in range(14):  
        gesture_folder = f'Gesture_{label}'
        gesture_path = os.path.join(root_path, 'HG14-Hand Gesture', gesture_folder)
        
        for dirname, _, filenames in os.walk(gesture_path):
            for filename in filenames:
                if filename.endswith('.jpg'):  
                    filepath = os.path.join(dirname, filename)
                    images.append(filepath)
                    labels.append(label)

    data = pd.DataFrame({'Image': images, 'Label': labels})
    return data

In [16]:
data = load_images_kaggle()

In [17]:
train_df, val_df = train_test_split(data, test_size=0.2, random_state=2024)

In [18]:
def preprocess_image(image):
    image = tf.image.resize(image, (224, 224)) 
    image = tf.keras.applications.resnet50.preprocess_input(image)
    return image

In [28]:
def create_dataset(df, batch_size=32, augment=False):
    image_paths = df['Image'].values
    labels = df['Label'].values

    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))
    dataset = dataset.map(lambda x, y: (tf.io.read_file(x), y))

    def decode_and_preprocess_image(image, label):
        image = tf.image.decode_jpeg(image, channels=3)
        image = preprocess_image(image)
        return image, label
    
    dataset = dataset.map(decode_and_preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    
    if augment:
        data_augmentation = tf.keras.Sequential([
            tf.keras.layers.RandomRotation(0.2),
            tf.keras.layers.RandomZoom(0.2),
            tf.keras.layers.RandomFlip("horizontal"),
            tf.keras.layers.RandomContrast(0.2),
        ])
        dataset = dataset.map(lambda x, y: (data_augmentation(x, training=True), y),
                              num_parallel_calls=tf.data.experimental.AUTOTUNE)

    dataset = dataset.shuffle(buffer_size=len(df))
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    return dataset

In [29]:
batch_size = 32
train_dataset = create_dataset(train_df, batch_size=batch_size, augment=True)  
val_dataset = create_dataset(val_df, batch_size=batch_size)

In [30]:
resnet50 = tf.keras.applications.ResNet50(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
for layer in resnet50.layers:
    layer.trainable = False

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [31]:
model_augment = models.Sequential([
    resnet50,
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation='relu'),
    layers.Dense(14, activation='softmax')
])

In [32]:
model_augment.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [33]:
epochs = 10
history = model_augment.fit(train_dataset,
                    epochs=epochs,
                    validation_data=val_dataset)

Epoch 1/10
  1/350 ━━━━━━━━━━━━━━━━━━━━ 9:40:23 100s/step - accuracy: 0.0625 - loss: 2.9951

I0000 00:00:1710688835.561471     158 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1710688835.617332     158 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


350/350 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - accuracy: 0.4718 - loss: 1.6071

W0000 00:00:1710688876.284895     158 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


350/350 ━━━━━━━━━━━━━━━━━━━━ 153s 153ms/step - accuracy: 0.4721 - loss: 1.6059 - val_accuracy: 0.6896 - val_loss: 0.8609
Epoch 2/10


W0000 00:00:1710688889.246467     157 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


350/350 ━━━━━━━━━━━━━━━━━━━━ 111s 115ms/step - accuracy: 0.7389 - loss: 0.7227 - val_accuracy: 0.7532 - val_loss: 0.7046
Epoch 3/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 110s 116ms/step - accuracy: 0.7871 - loss: 0.5823 - val_accuracy: 0.7200 - val_loss: 0.7778
Epoch 4/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 111s 117ms/step - accuracy: 0.8132 - loss: 0.5168 - val_accuracy: 0.7554 - val_loss: 0.6748
Epoch 5/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 111s 116ms/step - accuracy: 0.8328 - loss: 0.4636 - val_accuracy: 0.7936 - val_loss: 0.5626
Epoch 6/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 110s 116ms/step - accuracy: 0.8502 - loss: 0.4148 - val_accuracy: 0.7568 - val_loss: 0.7184
Epoch 7/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 109s 114ms/step - accuracy: 0.8495 - loss: 0.4065 - val_accuracy: 0.8068 - val_loss: 0.5715
Epoch 8/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 110s 116ms/step - accuracy: 0.8624 - loss: 0.3699 - val_accuracy: 0.7732 - val_loss: 0.7024
Epoch 9/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 111s 115ms/step - accuracy: 0.8675 - loss: 0.36

In [37]:
val_loss, val_accuracy = model_augment.evaluate(val_dataset)
formatted_accuracy = "{:.2f}%".format(val_accuracy * 100)
print("Accuracy on validation set after data augmentation:", formatted_accuracy)

88/88 ━━━━━━━━━━━━━━━━━━━━ 12s 82ms/step - accuracy: 0.8168 - loss: 0.5133
Accuracy on validation set after data augmentation: 82.29%


## 2.2 Transfer Learning

In [30]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split

In [31]:
def load_images_kaggle():
    images = []
    labels = []

    root_path = '/kaggle/input/hg14-handgesture14-dataset/HG14/'

    for label in range(14):  
        gesture_folder = f'Gesture_{label}'
        gesture_path = os.path.join(root_path, 'HG14-Hand Gesture', gesture_folder)
        
        for dirname, _, filenames in os.walk(gesture_path):
            for filename in filenames:
                if filename.endswith('.jpg'):  
                    filepath = os.path.join(dirname, filename)
                    images.append(filepath)
                    labels.append(label)

    data = pd.DataFrame({'Image': images, 'Label': labels})
    return data

In [32]:
data = load_images_kaggle()

In [33]:
train_df, val_df = train_test_split(data, test_size=0.2, random_state=2024)

In [8]:
def preprocess_image(image):
    image = tf.image.resize(image, (224, 224))  
    image = tf.keras.applications.resnet50.preprocess_input(image)
    return image

In [12]:
def create_dataset(df, batch_size=32):
    image_paths = df['Image'].values
    labels = df['Label'].values

    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))
    dataset = dataset.map(lambda x, y: (tf.io.read_file(x), y))

    def decode_and_preprocess_image(image, label):
        image = tf.image.decode_jpeg(image, channels=3)
        image = preprocess_image(image)
        return image, label
    
    dataset = dataset.map(decode_and_preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)

    dataset = dataset.shuffle(buffer_size=len(df))
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    return dataset

In [11]:
batch_size = 32
train_dataset = create_dataset(train_df, batch_size=batch_size)
val_dataset = create_dataset(val_df, batch_size=batch_size)

In [37]:
resnet50_base = tf.keras.applications.ResNet50(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
for layer in resnet50_base.layers:
    layer.trainable = False

In [38]:
transfer_model = models.Sequential([
    resnet50_base,
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation='relu'),
    layers.Dense(14, activation='softmax')
])

In [39]:
transfer_model.compile(optimizer='adam',
                       loss='sparse_categorical_crossentropy',
                       metrics=['accuracy'])

In [40]:
epochs = 10
history = transfer_model.fit(train_dataset,
                             epochs=epochs,
                             validation_data=val_dataset)

Epoch 1/10
  2/350 ━━━━━━━━━━━━━━━━━━━━ 40s 117ms/step - accuracy: 0.1016 - loss: 3.4744  

W0000 00:00:1710662581.549433      92 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


350/350 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - accuracy: 0.5582 - loss: 1.4362

W0000 00:00:1710662618.696957      92 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


350/350 ━━━━━━━━━━━━━━━━━━━━ 73s 131ms/step - accuracy: 0.5587 - loss: 1.4346 - val_accuracy: 0.8546 - val_loss: 0.4016
Epoch 2/10


W0000 00:00:1710662627.468156      92 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


350/350 ━━━━━━━━━━━━━━━━━━━━ 57s 115ms/step - accuracy: 0.8999 - loss: 0.2966 - val_accuracy: 0.8918 - val_loss: 0.3015
Epoch 3/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 57s 115ms/step - accuracy: 0.9546 - loss: 0.1566 - val_accuracy: 0.9211 - val_loss: 0.2214
Epoch 4/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 56s 115ms/step - accuracy: 0.9614 - loss: 0.1149 - val_accuracy: 0.9414 - val_loss: 0.1665
Epoch 5/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 56s 115ms/step - accuracy: 0.9754 - loss: 0.0827 - val_accuracy: 0.9468 - val_loss: 0.1514
Epoch 6/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 57s 116ms/step - accuracy: 0.9790 - loss: 0.0676 - val_accuracy: 0.9546 - val_loss: 0.1405
Epoch 7/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 82s 115ms/step - accuracy: 0.9826 - loss: 0.0533 - val_accuracy: 0.9443 - val_loss: 0.1760
Epoch 8/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 57s 115ms/step - accuracy: 0.9824 - loss: 0.0509 - val_accuracy: 0.9386 - val_loss: 0.1860
Epoch 9/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 57s 115ms/step - accuracy: 0.9829 - loss: 0.0565 - val

In [41]:
val_loss, val_accuracy = transfer_model.evaluate(val_dataset)
formatted_accuracy = "{:.2%}".format(val_accuracy)

print("Accuracy on validation set after transfer learning:", formatted_accuracy)

88/88 ━━━━━━━━━━━━━━━━━━━━ 11s 81ms/step - accuracy: 0.9380 - loss: 0.1960
Accuracy on validation set after transfer learning: 93.89%


## 2.3 Hyperparameter Tuning

In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split

2024-03-17 09:40:57.935174: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-17 09:40:57.935289: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-17 09:40:58.208868: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
def load_images_kaggle():
    images = []
    labels = []

    root_path = '/kaggle/input/hg14-handgesture14-dataset/HG14/'

    for label in range(14):  
        gesture_folder = f'Gesture_{label}'
        gesture_path = os.path.join(root_path, 'HG14-Hand Gesture', gesture_folder)
        
        for dirname, _, filenames in os.walk(gesture_path):
            for filename in filenames:
                if filename.endswith('.jpg'):  
                    filepath = os.path.join(dirname, filename)
                    images.append(filepath)
                    labels.append(label)

    data = pd.DataFrame({'Image': images, 'Label': labels})
    return data

In [3]:
data = load_images_kaggle()

In [4]:
train_df, val_df = train_test_split(data, test_size=0.2, random_state=2024)

In [5]:
def preprocess_image(image):
    image = tf.image.resize(image, (224, 224))  
    image = tf.keras.applications.resnet50.preprocess_input(image)
    return image

In [6]:
def create_dataset(df, batch_size=32):
    image_paths = df['Image'].values
    labels = df['Label'].values

    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))
    dataset = dataset.map(lambda x, y: (tf.io.read_file(x), y))

    def decode_and_preprocess_image(image, label):
        image = tf.image.decode_jpeg(image, channels=3)
        image = preprocess_image(image)
        return image, label
    
    dataset = dataset.map(decode_and_preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)

    dataset = dataset.shuffle(buffer_size=len(df))
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    return dataset

In [8]:
batch_size = 32
train_dataset = create_dataset(train_df, batch_size=batch_size)
val_dataset = create_dataset(val_df, batch_size=batch_size)

In [9]:
resnet50_base = tf.keras.applications.ResNet50(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
for layer in resnet50_base.layers:
    layer.trainable = False

In [13]:
def hyperparameter_tuning(optimizers, learning_rates):
    best_accuracy = 0.0
    best_lr = None
    best_optimizer = None

    for optimizer in optimizers:
        for lr in learning_rates:
            transfer_model = models.Sequential([
                resnet50_base,
                layers.GlobalAveragePooling2D(),
                layers.Dense(512, activation='relu'),
                layers.Dense(14, activation='softmax')
            ])

            transfer_model.compile(optimizer=optimizer(learning_rate=lr),
                                   loss='sparse_categorical_crossentropy',
                                   metrics=['accuracy'])

            history = transfer_model.fit(train_dataset, epochs=10, validation_data=val_dataset, verbose=0)

            val_loss, val_accuracy = transfer_model.evaluate(val_dataset)

            print(f"Learning Rate: {lr}, Optimizer: {optimizer.__name__}, Accuracy: {val_accuracy * 100:.2f}%")

            if val_accuracy > best_accuracy:
                best_accuracy = val_accuracy
                best_lr = lr
                best_optimizer = optimizer.__name__

    print(f"Best LR: {best_lr}, Best Optimizer: {best_optimizer}, Best Accuracy: {best_accuracy * 100:.2f}%")

optimizers = [tf.keras.optimizers.Adam, tf.keras.optimizers.RMSprop]
learning_rates = [0.001, 0.0001]

hyperparameter_tuning(optimizers, learning_rates)

I0000 00:00:1710668610.607359    1259 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1710668610.664020    1259 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1710668649.787745    1261 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1710668662.392025    1259 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


88/88 ━━━━━━━━━━━━━━━━━━━━ 11s 81ms/step - accuracy: 0.9508 - loss: 0.1447
Learning Rate: 0.001, Optimizer: Adam, Accuracy: 95.54%


W0000 00:00:1710669210.716157    1258 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1710669247.113945    1258 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1710669256.129348    1260 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


88/88 ━━━━━━━━━━━━━━━━━━━━ 12s 83ms/step - accuracy: 0.9562 - loss: 0.1539
Learning Rate: 0.0001, Optimizer: Adam, Accuracy: 95.39%


W0000 00:00:1710669808.073178    1259 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1710669845.052656    1261 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1710669854.031145    1260 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


88/88 ━━━━━━━━━━━━━━━━━━━━ 11s 81ms/step - accuracy: 0.9465 - loss: 0.2021
Learning Rate: 0.001, Optimizer: RMSprop, Accuracy: 94.82%


W0000 00:00:1710670400.926181    1260 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1710670437.387688    1260 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1710670446.239274    1259 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


88/88 ━━━━━━━━━━━━━━━━━━━━ 11s 83ms/step - accuracy: 0.9533 - loss: 0.1488
Learning Rate: 0.0001, Optimizer: RMSprop, Accuracy: 95.18%
Best LR: 0.001, Best Optimizer: Adam, Best Accuracy: 95.54%


# 3. Comparing Accuracy of Models

In [50]:
from tabulate import tabulate

data = [
    ["Base ResNet-50 Model", 94.75],
    ["Data Augmentation", 82.29],
    ["Transfer Learning", 93.89],
    ["Hyperparameter Tuning", 95.54]
]

headers = ["Model", "Accuracy (%)"]

table = tabulate(data, headers=headers, tablefmt="github")
print(table)

| Model                 |   Accuracy (%) |
|-----------------------|----------------|
| Base ResNet-50 Model  |          94.75 |
| Data Augmentation     |          82.29 |
| Transfer Learning     |          93.89 |
| Hyperparameter Tuning |          95.54 |


# 4. Evaluating Best Performing Model on Test Dataset 

In [63]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [64]:
def load_images_kaggle():
    images = []
    labels = []

    root_path = '/kaggle/input/hg14-handgesture14-dataset/HG14/'

    for label in range(14):  
        gesture_folder = f'Gesture_{label}'
        gesture_path = os.path.join(root_path, 'HG14-Hand Gesture', gesture_folder)
        
        for dirname, _, filenames in os.walk(gesture_path):
            for filename in filenames:
                if filename.endswith('.jpg'):  
                    filepath = os.path.join(dirname, filename)
                    images.append(filepath)
                    labels.append(label)

    data = pd.DataFrame({'Image': images, 'Label': labels})
    return data

data = load_images_kaggle()

In [65]:
train_val_df, test_df = train_test_split(data, test_size=0.2, random_state=2024)
train_df, val_df = train_test_split(train_val_df, test_size=0.2, random_state=2024)

In [66]:
def preprocess_image(image):
    image = tf.image.resize(image, (224, 224))  
    image = tf.keras.applications.resnet50.preprocess_input(image)
    return image

In [68]:
def create_dataset(df, batch_size=32):
    image_paths = df['Image'].values
    labels = df['Label'].values

    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))
    dataset = dataset.map(lambda x, y: (tf.io.read_file(x), y))

    def decode_and_preprocess_image(image, label):
        image = tf.image.decode_jpeg(image, channels=3)
        image = preprocess_image(image)
        return image, label
    
    dataset = dataset.map(decode_and_preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)

    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    return dataset

In [69]:
batch_size = 32
train_dataset = create_dataset(train_val_df, batch_size=batch_size)
val_dataset = create_dataset(val_df, batch_size=batch_size)
test_dataset = create_dataset(test_df, batch_size=batch_size)

In [70]:
resnet50_base = tf.keras.applications.ResNet50(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
for layer in resnet50_base.layers:
    layer.trainable = False

In [71]:
best_lr = 0.001
best_optimizer = tf.keras.optimizers.Adam(learning_rate=best_lr)

best_model = models.Sequential([
    resnet50_base,
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation='relu'),
    layers.Dense(14, activation='softmax')
])

In [72]:
best_model.compile(optimizer=best_optimizer,
                   loss='sparse_categorical_crossentropy',
                   metrics=['accuracy'])

In [73]:
epochs = 10
best_model.fit(train_dataset, epochs=epochs, validation_data=val_dataset, verbose=1)

Epoch 1/10
  1/350 ━━━━━━━━━━━━━━━━━━━━ 1:03:03 11s/step - accuracy: 0.0312 - loss: 3.6129

W0000 00:00:1710693221.362587     160 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


350/350 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - accuracy: 0.5338 - loss: 1.4619

W0000 00:00:1710693254.088605     158 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


350/350 ━━━━━━━━━━━━━━━━━━━━ 49s 110ms/step - accuracy: 0.5343 - loss: 1.4603 - val_accuracy: 0.9071 - val_loss: 0.3098
Epoch 2/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 34s 97ms/step - accuracy: 0.8956 - loss: 0.3171 - val_accuracy: 0.9460 - val_loss: 0.1725
Epoch 3/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 41s 97ms/step - accuracy: 0.9461 - loss: 0.1767 - val_accuracy: 0.9625 - val_loss: 0.1231
Epoch 4/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 41s 97ms/step - accuracy: 0.9621 - loss: 0.1218 - val_accuracy: 0.9812 - val_loss: 0.0691
Epoch 5/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 34s 97ms/step - accuracy: 0.9688 - loss: 0.0968 - val_accuracy: 0.9714 - val_loss: 0.0797
Epoch 6/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 34s 97ms/step - accuracy: 0.9656 - loss: 0.0971 - val_accuracy: 0.9759 - val_loss: 0.0717
Epoch 7/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 34s 97ms/step - accuracy: 0.9697 - loss: 0.0826 - val_accuracy: 0.9719 - val_loss: 0.0756
Epoch 8/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 34s 97ms/step - accuracy: 0.9770 - loss: 0.0645 - val_accura

In [74]:
test_loss, test_accuracy = best_model.evaluate(test_dataset)

88/88 ━━━━━━━━━━━━━━━━━━━━ 9s 102ms/step - accuracy: 0.9502 - loss: 0.1578


W0000 00:00:1710693598.883404     159 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


In [75]:
y_true = np.concatenate([y for _, y in test_dataset], axis=0)
y_pred_probs = best_model.predict(test_dataset)
y_pred = np.argmax(y_pred_probs, axis=1)

precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')

print("Test Accuracy: {:.2f}%".format(test_accuracy * 100))
print("Test Precision: {:.4f}".format(precision))
print("Test Recall: {:.4f}".format(recall))
print("Test F1 Score: {:.4f}".format(f1))

 3/88 ━━━━━━━━━━━━━━━━━━━━ 8s 99ms/step

W0000 00:00:1710693610.624136     159 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


88/88 ━━━━━━━━━━━━━━━━━━━━ 15s 120ms/step
Test Accuracy: 94.75%
Test Precision: 0.9492
Test Recall: 0.9475
Test F1 Score: 0.9472


W0000 00:00:1710693621.098086     157 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
